## Setup ResNet Model

In [25]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

num_classes = 4
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

## Split Data from Source

In [27]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Set up your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the entire dataset
dataset = ImageFolder(root='../data/corn-leaf-disease/', transform=transform)

In [37]:
dataset.classes

['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [28]:
from torch.utils.data import random_split

# Define the split ratio
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

### Create DataLoaders

In [29]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Training

In [30]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Epoch [1/10], Loss: 0.2968
Validation Accuracy: 84.49%
Epoch [2/10], Loss: 0.1987
Validation Accuracy: 89.86%
Epoch [3/10], Loss: 0.1512
Validation Accuracy: 92.60%
Epoch [4/10], Loss: 0.1190
Validation Accuracy: 94.75%
Epoch [5/10], Loss: 0.1098
Validation Accuracy: 94.27%
Epoch [6/10], Loss: 0.1318
Validation Accuracy: 89.98%
Epoch [7/10], Loss: 0.0813
Validation Accuracy: 92.96%
Epoch [8/10], Loss: 0.0961
Validation Accuracy: 93.56%
Epoch [9/10], Loss: 0.0670
Validation Accuracy: 94.87%
Epoch [10/10], Loss: 0.0607
Validation Accuracy: 84.13%


In [33]:
from PIL import Image

def predict_image(image_path, model, transform, device):
    model.eval()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# Example usage
image_path = '../data/corn-leaf-disease/Blight/Corn_Blight (1).jpg'
prediction = predict_image(image_path, model, transform, device)
print(f'Predicted Class: {prediction}')

Predicted Class: 2


In [39]:
from PIL import Image

def predict_image(image_path, model, transform, device, class_names):
    model.eval()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    predicted_class = class_names[predicted.item()]
    return predicted_class

# Assuming your dataset is already loaded
dataset = ImageFolder(root='../data/corn-leaf-disease', transform=transform)
class_names = dataset.classes  # Get class names from the dataset

# Example usage
image_path = '../data/corn-leaf-disease/Common_Rust/Corn_Common_Rust (7).jpg'
prediction = predict_image(image_path, model, transform, device, class_names)
print(f'Predicted Class: {prediction}')


Predicted Class: Common_Rust


In [47]:
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 4)
torch.save(model.state_dict(), "resnet50_corn_disease.pth")

In [49]:
import torch
import torchvision.models as models

# Initialize the model
model = models.resnet50(weights=None)

# Modify the final fully connected layer to match the number of classes (e.g., 4)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 4)

# Load the trained model’s state_dict
model.load_state_dict(torch.load("resnet50_corn_disease.pth"))

# Set the model to evaluation mode
model.eval()

C:\Users\borad\AppData\Local\Temp\ipykernel_26516\2218050575.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("resnet50_corn_disease.pth

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [56]:
image_path = '../data/corn-leaf-disease/Common_Rust/Corn_Common_Rust (12).jpg'
prediction = predict_image(image_path, model.to("cuda"), transform, device, class_names)
print(f'Predicted Class: {prediction}')

Predicted Class: Common_Rust
